Se transforman los datos desde Stata a CSV

In [15]:
import pandas as pd

# Leer sin convertir categorías
df = pd.read_stata("BaseCasen2022.dta", convert_categoricals=False)

reader = pd.io.stata.StataReader("BaseCasen2022.dta")
value_labels = reader.value_labels()
reader.close()

# Guardar
df.to_csv("CASEN_2022.csv", index=False, encoding='utf-8-sig')
print(f"¡Conversión completa! {len(df)} filas guardadas")

C:\Users\Liz\AppData\Local\Temp\ipykernel_15028\1338345894.py:4: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  df = pd.read_stata("BaseCasen2022.dta", convert_categoricals=False)
C:\Users\Liz\AppData\Local\Temp\ipykernel_15028\1338345894.py:7: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  value_labels = reader.value_labels()
C:\Users\Liz\AppData\Local\Temp\ipykernel_15028\1338345894.py:8: FutureWarning: The StataReader.close() method is not part of the public API and will be removed in a future versio

¡Conversión completa! 202231 filas guardadas


Inspeccionar los datos del cansen para evaluarlos

In [16]:
df.head()



,id_vivienda,folio,id_persona,region,area,cod_upm,nse,estrato,hogar,expr,...,men18c,may60c,tipohogar,tot_hog,ind_hacina,indsan,ten_viv,ten_viv_f,allega_ext,allega_int
0,1000901,100090101,1,16,2,10009,4,1630324,1,43,...,0,1,3,1,1,1,1,1,0,0
1,1000901,100090101,2,16,2,10009,4,1630324,1,43,...,0,1,3,1,1,1,1,1,0,0
2,1000901,100090101,3,16,2,10009,4,1630324,1,44,...,0,1,3,1,1,1,1,1,0,0
3,1000902,100090201,1,16,2,10009,4,1630324,1,51,...,1,1,5,1,1,2,1,1,0,1
4,1000902,100090201,2,16,2,10009,4,1630324,1,51,...,1,1,5,1,1,2,1,1,0,1


Ver datos de pobreza

In [17]:
print(df['pobreza'].value_counts())

pobreza
3.0    186838
2.0     10616
1.0      4657
Name: count, dtype: int64


In [18]:
print('pobreza' in df.columns)

True


Ya investigado la documentacion entregada por la web de la base de datos y el dataframe, tenemos el nombre de las variables para crear un dataframe de pobreza con CASEN 2022

In [31]:
# Variables correctas disponibles
variables_necesarias = ['folio', 'id_persona', 'pobreza', 'pobreza_multi_5d','lp','li',
                        'nse', 'region', 'edad', 'sexo', 'ytotcorh','yoprcorh', 'expr']

df_pobreza = df[variables_necesarias].copy()

# Eliminar filas donde pobreza sea nula
df_pobreza = df_pobreza.dropna(subset=['pobreza'])

# Ver distribución de pobreza
print(" DISTRIBUCIÓN DE POBREZA POR INGRESOS ")
print(df_pobreza['pobreza'].value_counts().sort_index())

print("\n DISTRIBUCIÓN DE POBREZA MULTIDIMENSIONAL ")
print(df_pobreza['pobreza_multi_5d'].value_counts().sort_index())

print("\n DISTRIBUCIÓN DE NIVEL SOCIOECONÓMICO (NSE) ")
print(df_pobreza['nse'].value_counts().sort_index())

print(f"\nTotal personas con datos: {len(df_pobreza)}")

 DISTRIBUCIÓN DE POBREZA POR INGRESOS 
pobreza
1.0      4657
2.0     10616
3.0    186838
Name: count, dtype: int64

 DISTRIBUCIÓN DE POBREZA MULTIDIMENSIONAL 
pobreza_multi_5d
0.0    161671
1.0     37254
Name: count, dtype: int64

 DISTRIBUCIÓN DE NIVEL SOCIOECONÓMICO (NSE) 
nse
1    40908
2    50198
3    19889
4    55990
5      261
6    19073
7    15792
Name: count, dtype: int64

Total personas con datos: 202111


In [32]:
df_pobreza.info()

<class 'pandas.core.frame.DataFrame'>
Index: 202111 entries, 0 to 202230
Data columns (total 13 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   folio             202111 non-null  int32  
 1   id_persona        202111 non-null  int8   
 2   pobreza           202111 non-null  float64
 3   pobreza_multi_5d  198925 non-null  float64
 4   lp                202111 non-null  int32  
 5   li                202111 non-null  int32  
 6   nse               202111 non-null  int8   
 7   region            202111 non-null  int8   
 8   edad              202111 non-null  int16  
 9   sexo              202111 non-null  int8   
 10  ytotcorh          202111 non-null  float64
 11  yoprcorh          202111 non-null  int32  
 12  expr              202111 non-null  int16  
dtypes: float64(3), int16(2), int32(4), int8(4)
memory usage: 10.8 MB


Limpiamos los datos nulos para 'pobreza_multi_5d'

In [33]:
df_pobreza = df_pobreza.dropna(subset=['pobreza_multi_5d'])

In [34]:
df_pobreza.head()

,folio,id_persona,pobreza,pobreza_multi_5d,lp,li,nse,region,edad,sexo,ytotcorh,yoprcorh,expr
0,100090101,1,3.0,0.0,216849,144566,4,16,72,2,1010894.0,411242,43
1,100090101,2,3.0,0.0,216849,144566,4,16,67,1,1010894.0,411242,43
2,100090101,3,3.0,0.0,216849,144566,4,16,40,2,1010894.0,411242,44
3,100090201,1,2.0,1.0,216849,144566,4,16,56,1,418192.0,0,51
4,100090201,2,2.0,1.0,216849,144566,4,16,25,2,418192.0,0,51


In [35]:
df_pobreza.info()

<class 'pandas.core.frame.DataFrame'>
Index: 198925 entries, 0 to 202230
Data columns (total 13 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   folio             198925 non-null  int32  
 1   id_persona        198925 non-null  int8   
 2   pobreza           198925 non-null  float64
 3   pobreza_multi_5d  198925 non-null  float64
 4   lp                198925 non-null  int32  
 5   li                198925 non-null  int32  
 6   nse               198925 non-null  int8   
 7   region            198925 non-null  int8   
 8   edad              198925 non-null  int16  
 9   sexo              198925 non-null  int8   
 10  ytotcorh          198925 non-null  float64
 11  yoprcorh          198925 non-null  int32  
 12  expr              198925 non-null  int16  
dtypes: float64(3), int16(2), int32(4), int8(4)
memory usage: 10.6 MB


In [36]:
df_pobreza.to_csv("CASEN_POBREZA_2022.csv", index=False, encoding='utf-8-sig')

Ahora uniremos los archivos los dataframes de CASEN_POBREZA_2022.csv y CASEN_GEOGRAFICO_2022.csv, para crear un nuevo dataframe de CASEN_POBREZA_GEOGRAFICA_2022.csv

In [38]:
df_geografico = pd.read_csv("CASEN_GEOGRAFICO_2022.csv")


In [40]:
df_completo = pd.merge(
    df_pobreza,
    df_geografico,
    on=['folio', 'id_persona'],
    how='left'
)

# Verificar el resultado
print(f"Filas en pobreza: {len(df_pobreza)}")
print(f"Filas en geográfico: {len(df_geografico)}")
print(f"Filas después de unir: {len(df_completo)}")

Filas en pobreza: 198925
Filas en geográfico: 202231
Filas después de unir: 198925


In [41]:
df_completo.head()

,folio,id_persona,pobreza,pobreza_multi_5d,lp,li,nse,region,edad,sexo,ytotcorh,yoprcorh,expr,provincia,comuna,expp,expc
0,100090101,1,3.0,0.0,216849,144566,4,16,72,2,1010894.0,411242,43,163,16303,51,52
1,100090101,2,3.0,0.0,216849,144566,4,16,67,1,1010894.0,411242,43,163,16303,50,53
2,100090101,3,3.0,0.0,216849,144566,4,16,40,2,1010894.0,411242,44,163,16303,51,52
3,100090201,1,2.0,1.0,216849,144566,4,16,56,1,418192.0,0,51,163,16303,51,52
4,100090201,2,2.0,1.0,216849,144566,4,16,25,2,418192.0,0,51,163,16303,50,52


In [42]:
df_completo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198925 entries, 0 to 198924
Data columns (total 17 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   folio             198925 non-null  int32  
 1   id_persona        198925 non-null  int8   
 2   pobreza           198925 non-null  float64
 3   pobreza_multi_5d  198925 non-null  float64
 4   lp                198925 non-null  int32  
 5   li                198925 non-null  int32  
 6   nse               198925 non-null  int8   
 7   region            198925 non-null  int8   
 8   edad              198925 non-null  int16  
 9   sexo              198925 non-null  int8   
 10  ytotcorh          198925 non-null  float64
 11  yoprcorh          198925 non-null  int32  
 12  expr              198925 non-null  int16  
 13  provincia         198925 non-null  int64  
 14  comuna            198925 non-null  int64  
 15  expp              198925 non-null  int64  
 16  expc              19

Ya creado lo guardamos en un csv

In [43]:
df.to_csv('CASEN_POBREZA_GEOGRAFICA_2022.csv', index=False, encoding='utf-8-sig')